# Ensemble Learning

## Initial Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [5]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [14]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)[:-2]

In [22]:
columns = list(df.columns)

In [23]:
target = ['loan_status']

In [24]:
# Load the data
df = df.loc[:, columns].copy()

# Drop null columns
df = df.dropna(axis="columns", how="all")

# Drop null rows
df = df.dropna()

# remove issued loans
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

In [25]:
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [27]:
# conv target column values to low_risk and high_risk based on their values 
x = {'Current': 'low_risk'}
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30)', 'Default', 'In Grace Period'], 'high_risk')
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [28]:
# Create our features
X = df.drop("loan_status", axis=1)

X = pd.get_dummies(X)

# Create our target
y = df["loan_status"]

In [29]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68815.000000,68815.000000,68815.000000,6.881500e+04,68815.000000,68815.000000,68815.000000,68815.000000,68815.000000,68815.000000,...,68815.000000,68815.0,68815.000000,68815.000000,68815.000000,68815.000000,68815.000000,68815.000000,68815.0,68815.0
mean,16677.265494,0.127719,480.643936,8.820767e+04,21.778539,0.217743,0.497697,12.587227,0.126019,17603.616741,...,0.177243,1.0,0.123868,0.876132,0.383172,0.616828,0.860336,0.139664,1.0,1.0
std,10277.113051,0.048130,288.052205,1.155699e+05,20.199409,0.718345,0.758128,6.022871,0.336785,21835.601935,...,0.381877,0.0,0.329434,0.329434,0.486163,0.486163,0.346641,0.346641,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.895000,0.000000,0.000000,8.000000,0.000000,6292.500000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,647.950000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21734.500000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [30]:
# Check the balance of our target values
len(X.columns)

95

In [31]:
# Split the X and y into X_train, X_test, y_train, y_test
y.value_counts()

low_risk     68468
high_risk      347
Name: loan_status, dtype: int64

In [32]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1,
                                                    stratify=y)

X_train.shape

(51611, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [35]:
# Create the StandardScaler instance
from sklearn.preprocessing import LabelEncoder, StandardScaler
scaler = StandardScaler()

In [36]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [37]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [38]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators = 100)
random_forest = random_forest.fit(X_train, y_train)

In [39]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.811366531491513

In [40]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   62,    25],
       [ 1539, 15578]])

In [42]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.71      0.91      0.07      0.81      0.64        87
   low_risk       1.00      0.91      0.71      0.95      0.81      0.66     17117

avg / total       0.99      0.91      0.71      0.95      0.81      0.66     17204



In [43]:
# List the features sorted in descending order by feature importance
feature_names = X.columns
sorted(zip(random_forest.feature_importances_, feature_names), reverse=True)

[(0.07109649256475065, 'total_rec_prncp'),
 (0.06563226501357299, 'total_rec_int'),
 (0.05834369764841581, 'last_pymnt_amnt'),
 (0.05820572266936268, 'total_pymnt'),
 (0.05532346536536668, 'total_pymnt_inv'),
 (0.027682745919149356, 'int_rate'),
 (0.02301951663906621, 'issue_d_Jan-2019'),
 (0.020217216488807033, 'installment'),
 (0.01700992537785736, 'annual_inc'),
 (0.016603784514202843, 'out_prncp'),
 (0.01655463365439887, 'tot_hi_cred_lim'),
 (0.015983572955456694, 'mths_since_recent_inq'),
 (0.01561821597376751, 'out_prncp_inv'),
 (0.015544871495618924, 'issue_d_Mar-2019'),
 (0.015378107527259342, 'dti'),
 (0.015302364324902536, 'max_bal_bc'),
 (0.015036828979194365, 'mo_sin_old_il_acct'),
 (0.014980155087404044, 'bc_open_to_buy'),
 (0.014798257202789772, 'total_rev_hi_lim'),
 (0.014554267611091687, 'mo_sin_old_rev_tl_op'),
 (0.01448284110104619, 'bc_util'),
 (0.01423095218211955, 'total_bc_limit'),
 (0.013966295050798182, 'tot_cur_bal'),
 (0.013677988988807092, 'loan_amnt'),
 (0.0

### Easy Ensemble Classifier

In [44]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
ensemble = EasyEnsembleClassifier(n_estimators = 100, random_state=1)
ensemble = ensemble.fit(X_train, y_train)

In [45]:
# Calculated the balanced accuracy score
y_pred = ensemble.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9352257183320474

In [46]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   81,     6],
       [ 1037, 16080]])

In [47]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.93      0.94      0.13      0.94      0.87        87
   low_risk       1.00      0.94      0.93      0.97      0.94      0.88     17117

avg / total       0.99      0.94      0.93      0.96      0.94      0.88     17204



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifier had a higher balanced accuracy score.  The Balanced Random Forest Classifier was still a fairly good balanced score of .81.  

2. Which model had the best recall score?

    The Easy Ensemble Learner had the better score here as well.  

3. Which model had the best geometric mean score?

    The Easy Ensemble Learner had the better score here as well. 

4. What are the top three features?

    YOUR ANSWER HERE.